In [2]:
import pandas as pd
import numpy as np
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import os
from torchinfo import summary
import time

# Thiết lập môi trường
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

# Định nghĩa các case tham số
cases = {
    1: {'sequence_length': 28, 'batch_size': 32, 'num_epochs': 20, 'patience': 5, 'dropout': 0.3, 'hidden_size': 128},
    2: {'sequence_length': 56, 'batch_size': 32, 'num_epochs': 20, 'patience': 5, 'dropout': 0.3, 'hidden_size': 128},
    3: {'sequence_length': 56, 'batch_size': 32, 'num_epochs': 30, 'patience': 5, 'dropout': 0.3, 'hidden_size': 128},
    4: {'sequence_length': 28, 'batch_size': 32, 'num_epochs': 30, 'patience': 3, 'dropout': 0.2, 'hidden_size': 128},
    5: {'sequence_length': 28, 'batch_size': 32, 'num_epochs': 30, 'patience': 5, 'dropout': 0.1, 'hidden_size': 128},
}

# Bước 1: Tải dữ liệu từ SQLite
def load_data_from_sqlite():
    conn = sqlite3.connect('historical_data1.db')
    query = "SELECT * FROM historical_data1 ORDER BY item_id, store_id, date"
    data = pd.read_sql_query(query, conn)
    conn.close()
    data['date'] = pd.to_datetime(data['date'])
    print("Data loaded from SQLite successfully!")
    print(f"Data shape: {data.shape}")
    print("Checking for NaN in features...")
    print(data.isna().sum())
    return data

# Dataset và DataLoader
class SalesDataset(Dataset):
    def __init__(self, X, y, info=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.info = info
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.info is not None:
            return self.X[idx], self.y[idx], self.info[idx]
        return self.X[idx], self.y[idx]

def prepare_lstm_data(data, features, sequence_length, batch_size):
    X, y, info = [], [], []
    unique_items = data[['item_id', 'store_id']].drop_duplicates().values
    
    with open('utils/scaler1.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    valid_pairs = 0
    for item_id, store_id in unique_items:
        item_data = data[(data['item_id'] == item_id) & (data['store_id'] == store_id)].sort_values('date')
        if len(item_data) < sequence_length + 1:
            continue
        valid_pairs += 1
        item_features = item_data[features].values
        item_dates = item_data['date'].astype(str).values
        num_samples = len(item_data) - sequence_length
        for i in range(num_samples):
            X.append(item_features[i:i + sequence_length])
            y_val = item_features[i + sequence_length, 0]  # Giá trị gốc của sales
            # Chuẩn hóa y_val bằng scaler với DataFrame
            df_to_scale = pd.DataFrame([[y_val] + [0] * (len(scaler.feature_names_in_) - 1)], columns=scaler.feature_names_in_)
            y_scaled = scaler.transform(df_to_scale)[:, 0]
            y.append(y_scaled)
            info.append((item_id, store_id, item_dates[i + sequence_length]))
    
    X = np.array(X)
    y = np.array(y).flatten()
    info = np.array(info, dtype=object)
    
    print(f"y trước chuẩn hóa (một số mẫu):", item_features[:5, 0])  # Kiểm tra giá trị gốc
    print(f"y sau chuẩn hóa (một số mẫu):", y[:5])  # Kiểm tra giá trị chuẩn hóa
    
    train_size = int(0.7 * len(X))
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    info_val = info[train_size:].tolist()
    
    print(f"Total samples created: X={len(X)}, y={len(y)}, info={len(info)}")
    print(f"Train samples: X_train={len(X_train)}, y_train={len(y_train)}")
    print(f"Validation samples: X_val={len(X_val)}, y_val={len(y_val)}, info_val={len(info_val)}")
    if not (len(X_val) == len(y_val) == len(info_val)):
        raise ValueError(f"Mismatch in validation data: X_val={len(X_val)}, y_val={len(y_val)}, info_val={len(info_val)}")
    
    train_dataset = SalesDataset(X_train, y_train)
    val_dataset = SalesDataset(X_val, y_val, info_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    print("LSTM data prepared successfully!")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_val shape: {X_val.shape}")
    print(f"Length of info_val: {len(info_val)}")
    return train_loader, val_loader, X_val, y_val, info_val

# Mô hình LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size=12, hidden_size=128, num_layers=3, dropout=0.3):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        out = torch.clamp(out, 0, 1)  # Giới hạn đầu ra trong [0, 1] để phù hợp với MinMaxScaler
        return out

def print_model_summary(model, input_size):
    summary(model, input_size=input_size)

# Hàm huấn luyện (giả sử bạn có hàm train_model, tôi sẽ để trống)
def train_model(model, train_loader, val_loader, num_epochs, patience, scaler, case_name):
    # Thêm code huấn luyện của bạn ở đây
    pass

# Hàm tính WRMSSE (giả sử bạn có hàm calculate_wrmsse)
def calculate_wrmsse(val_df):
    # Thêm code tính WRMSSE của bạn ở đây
    return 0.8814  # Giá trị giả định

# Hàm chính cho huấn luyện và đánh giá
def main_train():
    torch.cuda.empty_cache()
    
    data = load_data_from_sqlite()
    
    with open('utils/scaler1.pkl', 'rb') as f:
        scaler = pickle.load(f)
    print("Scaler feature names:", scaler.feature_names_in_)
    print("Scaler data_min:", scaler.data_min_)
    print("Scaler data_max:", scaler.data_max_)
    print("Scaler scale (computed):", scaler.data_max_ - scaler.data_min_)
    
    features = ['sales', 'sell_price', 'day_of_week', 'snap_CA', 'is_holiday', 'month', 'day_of_month',
                'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'sales_roll_mean_7', 'sales_roll_mean_14']
    
    print("\nCác case tham số có sẵn:")
    for case_num, params in cases.items():
        print(f"Case {case_num}: {params}")
    print("Nhập '0' để tự định nghĩa case mới.")
    
    case_choice = int(input("Nhập số case (0-5): "))
    
    if case_choice == 0:
        params = {
            'sequence_length': int(input("Nhập sequence_length: ")),
            'batch_size': int(input("Nhập batch_size: ")),
            'num_epochs': int(input("Nhập num_epochs: ")),
            'patience': int(input("Nhập patience: ")),
            'dropout': float(input("Nhập dropout (0.0-1.0): ")),
            'hidden_size': int(input("Nhập hidden_size: "))
        }
        case_name = f"lstm_model_custom_{params['sequence_length']}_{params['batch_size']}_{params['hidden_size']}"
    else:
        params = cases.get(case_choice, cases[1])
        case_name = f"lstm_model_{case_choice}"
    
    print(f"Đang chạy với tham số: {params}")
    print(f"Tên mô hình: {case_name}")
    
    train_loader, val_loader, X_val, y_val, info_val = prepare_lstm_data(
        data, features, params['sequence_length'], params['batch_size']
    )
    
    print("Kiểm tra dữ liệu trước khi huấn luyện...")
    if not (len(X_val) == len(y_val) == len(info_val)):
        raise ValueError(f"Dữ liệu validation không đồng bộ: X_val={len(X_val)}, y_val={len(y_val)}, info_val={len(info_val)}")
    
    model = LSTMModel(
        input_size=len(features),
        hidden_size=params['hidden_size'],
        dropout=params['dropout']
    ).to(device)
    print_model_summary(model, (params['sequence_length'], len(features)))
    
    model, metrics = train_model(model, train_loader, val_loader, params['num_epochs'], params['patience'], scaler, case_name)
    
    y_pred, rmse, mae, wrmsse = evaluate_model(model, X_val, y_val, info_val, scaler, params['batch_size'])
    
    print("\nHuấn luyện và đánh giá hoàn tất!")
    
    final_model_path = f"{case_name}.pth"
    torch.save(model.state_dict(), final_model_path)
    print(f"Mô hình cuối cùng đã được lưu tại {final_model_path}")
    
    return {
        'model': model,
        'y_pred': y_pred,
        'y_val': y_val,
        'info_val': info_val,
        'scaler': scaler,
        'rmse': rmse,
        'mae': mae,
        'wrmsse': wrmsse,
        'case_name': case_name
    }

def evaluate_model(model, X_val, y_val, info_val, scaler, batch_size):
    model.eval()
    dataset = SalesDataset(X_val, y_val, info_val)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    y_pred_list = []
    y_true_list = []
    info_list = []
    current_idx = 0
    
    with torch.no_grad():
        for X_batch, y_batch, _ in data_loader:
            batch_size_actual = X_batch.shape[0]
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            y_pred_batch = outputs.squeeze().cpu().numpy()
            y_true_batch = y_batch.numpy()
            batch_info = info_val[current_idx:current_idx + batch_size_actual]
            y_pred_list.append(y_pred_batch)
            y_true_list.append(y_true_batch)
            info_list.extend(batch_info)
            current_idx += batch_size_actual
    
    y_pred = np.concatenate(y_pred_list)
    y_true = np.concatenate(y_true_list)
    
    print("Length of info_list:", len(info_list))
    print("Length of y_true:", len(y_true))
    print("Length of y_pred:", len(y_pred))
    
    if not (len(info_list) == len(y_true) == len(y_pred)):
        raise ValueError(f"Mismatch in lengths: info_list={len(info_list)}, y_true={len(y_true)}, y_pred={len(y_pred)}")
    
    print("y_true (chuẩn hóa):", y_true[:10])
    print("y_pred (chuẩn hóa):", y_pred[:10])
    
    # Chuyển đổi ngược với MinMaxScaler sử dụng DataFrame
    y_true_df = pd.DataFrame(np.zeros((len(y_true), len(scaler.feature_names_in_))), columns=scaler.feature_names_in_)
    y_true_df['sales'] = y_true
    y_pred_df = pd.DataFrame(np.zeros((len(y_pred), len(scaler.feature_names_in_))), columns=scaler.feature_names_in_)
    y_pred_df['sales'] = y_pred
    
    y_true_orig = scaler.inverse_transform(y_true_df)[:, 0]
    y_pred_orig = scaler.inverse_transform(y_pred_df)[:, 0]
    
    print("y_true_orig (sau inverse):", y_true_orig[:10])
    print("y_pred_orig (sau inverse):", y_pred_orig[:10])
    
    rmse_scaled = np.sqrt(mean_squared_error(y_true, y_pred))
    mae_scaled = mean_absolute_error(y_true, y_pred)
    
    rmse_orig = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
    mae_orig = mean_absolute_error(y_true_orig, y_pred_orig)
    
    val_df = pd.DataFrame({
        'item_id': [info[0] for info in info_list],
        'store_id': [info[1] for info in info_list],
        'date': [info[2] for info in info_list],
        'actual': y_true_orig,
        'predicted': y_pred_orig
    })
    wrmsse = calculate_wrmsse(val_df)
    
    print("\nChỉ số trên dữ liệu chuẩn hóa:")
    print(f"RMSE (chuẩn hóa): {rmse_scaled:.6f}")
    print(f"MAE (chuẩn hóa): {mae_scaled:.6f}")
    print("\nChỉ số trên dữ liệu gốc:")
    print(f"RMSE (gốc): {rmse_orig:.4f}")
    print(f"MAE (gốc): {mae_orig:.4f}")
    print(f"WRMSSE: {wrmsse:.4f}")
    
    return y_pred_orig, rmse_orig, mae_orig, wrmsse

if __name__ == "__main__":
    results = main_train()
    # Gán các biến để sử dụng trong cell vẽ biểu đồ
    model = results['model']
    y_pred = results['y_pred']
    y_val = results['y_val']
    info_val = results['info_val']
    scaler = results['scaler']
    rmse = results['rmse']
    mae = results['mae']
    wrmsse = results['wrmsse']
    case_name = results['case_name']

Đang sử dụng thiết bị: cuda
Data loaded from SQLite successfully!
Data shape: (304920, 15)
Checking for NaN in features...
item_id               0
store_id              0
date                  0
sales                 0
sell_price            0
day_of_week           0
snap_CA               0
is_holiday            0
month                 0
day_of_month          0
sales_lag_7           0
sales_lag_14          0
sales_lag_28          0
sales_roll_mean_7     0
sales_roll_mean_14    0
dtype: int64
Scaler feature names: ['sales' 'sell_price' 'day_of_week' 'snap_CA' 'is_holiday' 'month'
 'day_of_month' 'sales_lag_7' 'sales_lag_14' 'sales_lag_28'
 'sales_roll_mean_7' 'sales_roll_mean_14']
Scaler data_min: [0.         0.09997559 0.         0.         0.         1.
 1.         0.         0.         0.         0.         0.        ]
Scaler data_max: [763.           4.98046875   6.           1.           1.
  12.          31.         763.         763.         763.
 602.85714286 468.64285714]
Scaler 

c:\Users\Ho Hau\Downloads\M5\venv\Lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Hàm để vẽ biểu đồ so sánh dữ liệu thực tế và dự đoán
def plot_actual_vs_predicted(y_true, y_pred, info_val, scaler, rmse, mae, wrmsse, case_name, num_samples=100, item_id=None, store_id=None):
    """
    Vẽ biểu đồ so sánh dữ liệu thực tế và dự đoán trên tập validation.
    
    Parameters:
    - y_true: Giá trị thực tế (chuẩn hóa) từ tập validation.
    - y_pred: Giá trị dự đoán (chuẩn hóa) từ mô hình.
    - info_val: Thông tin về item_id, store_id, và date cho tập validation.
    - scaler: Scaler để chuyển đổi ngược về giá trị gốc.
    - rmse: RMSE trên giá trị gốc.
    - mae: MAE trên giá trị gốc.
    - wrmsse: WRMSSE trên giá trị gốc.
    - case_name: Tên case để hiển thị trong tiêu đề và lưu file.
    - num_samples: Số mẫu tối đa để vẽ (mặc định: 100).
    - item_id: Nếu được cung cấp, chỉ vẽ cho item_id cụ thể.
    - store_id: Nếu được cung cấp, chỉ vẽ cho store_id cụ thể.
    """
    # Chuyển đổi ngược với MinMaxScaler sử dụng DataFrame
    y_true_df = pd.DataFrame(np.zeros((len(y_true), len(scaler.feature_names_in_))), columns=scaler.feature_names_in_)
    y_true_df['sales'] = y_true
    y_pred_df = pd.DataFrame(np.zeros((len(y_pred), len(scaler.feature_names_in_))), columns=scaler.feature_names_in_)
    y_pred_df['sales'] = y_pred
    
    y_true_orig = scaler.inverse_transform(y_true_df)[:, 0]
    y_pred_orig = scaler.inverse_transform(y_pred_df)[:, 0]
    
    # Kiểm tra NaN hoặc inf
    if np.any(np.isnan(y_true_orig)) or np.any(np.isnan(y_pred_orig)) or np.any(np.isinf(y_true_orig)) or np.any(np.isinf(y_pred_orig)):
        print("Cảnh báo: Dữ liệu chứa NaN hoặc inf sau khi chuyển đổi ngược!")
        return
    
    # Tạo DataFrame từ info_val để dễ lọc
    val_df = pd.DataFrame({
        'item_id': [info[0] for info in info_val],
        'store_id': [info[1] for info in info_val],
        'date': [info[2] for info in info_val],
        'actual': y_true_orig,
        'predicted': y_pred_orig
    })
    
    # Lọc theo item_id và store_id nếu được cung cấp
    if item_id is not None and store_id is not None:
        val_df = val_df[(val_df['item_id'] == item_id) & (val_df['store_id'] == store_id)]
        if val_df.empty:
            print(f"Không có dữ liệu cho item_id={item_id} và store_id={store_id} trong tập validation.")
            return
        title = f'So sánh thực tế vs dự đoán cho Item {item_id}, Store {store_id} (Case {case_name})'
    else:
        # Lấy một số mẫu đầu tiên nếu không lọc
        val_df = val_df.iloc[:min(num_samples, len(val_df))]
        title = f'So sánh thực tế vs dự đoán trên {len(val_df)} mẫu đầu tiên (Case {case_name})'
    
    # Sắp xếp theo date để vẽ theo thứ tự thời gian
    val_df['date'] = pd.to_datetime(val_df['date'], utc=True, errors='coerce')
    val_df = val_df.sort_values('date')
    
    # Loại bỏ NaN nếu có sau khi chuyển đổi datetime
    val_df = val_df.dropna()
    
    # Vẽ biểu đồ
    plt.figure(figsize=(14, 8))  # Tăng kích thước biểu đồ
    plt.plot(val_df['date'], val_df['actual'], label='Thực tế', color='blue', marker='o', linestyle='-')
    plt.plot(val_df['date'], val_df['predicted'], label='Dự đoán', color='red', marker='x', linestyle='--')
    
    # Thêm tiêu đề và nhãn
    plt.title(f'{title}\nRMSE: {rmse:.4f}, MAE: {mae:.4f}, WRMSSE: {wrmsse:.4f}', fontsize=10)
    plt.xlabel('Ngày', fontsize=10)
    plt.ylabel('Doanh số (giá trị gốc)', fontsize=10)
    plt.legend(fontsize=10)
    plt.grid(True)
    
    # Xoay nhãn ngày để dễ đọc
    plt.xticks(rotation=45, fontsize=8)
    
    # Điều chỉnh layout để tránh cắt nhãn
    plt.tight_layout()
    
    # Lưu biểu đồ
    plt.savefig(f'actual_vs_predicted_{case_name}.png', dpi=300, bbox_inches='tight')
    
    # Hiển thị biểu đồ
    plt.show()

# Kiểm tra các item_id và store_id trong tập validation
unique_items = set((info[0], info[1]) for info in info_val)
print("Các cặp item_id, store_id trong tập validation:", unique_items)

# Vẽ biểu đồ cho HOUSEHOLD_1465, CA_2
plot_actual_vs_predicted(y_val, y_pred, info_val, scaler, rmse, mae, wrmsse, case_name, item_id='HOUSEHOLD_1465', store_id='CA_2')